In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import pandas as pd
import ast
import math
from binary_utils import bin_decoder, bin_encoder, int_to_binary_str, binary_str_to_int

In [88]:
data = pd.read_csv('./data/eqs.csv')

In [89]:
y = data["eqs"]

In [90]:
X = data.drop('eqs', axis = 1)

In [91]:
eq = y[0]

In [92]:
[int_to_binary_str(x, 6) for x in X.head(1).to_numpy()[0]]

['0000000',
 '0011000',
 '0011100',
 '0100000',
 '0100100',
 '0101000',
 '0101100',
 '0110000',
 '0110100',
 '0111000']

In [95]:
seq = [torch.Tensor(bin_encoder(int_to_binary_str(x, 6))) for x in X.head(1).to_numpy()[0]]

In [96]:
seq

[tensor([0., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 1., 1., 0., 0., 0.]),
 tensor([0., 0., 1., 1., 1., 0., 0.]),
 tensor([0., 1., 0., 0., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 1., 0., 0.]),
 tensor([0., 1., 0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 1., 1., 0., 0.]),
 tensor([0., 1., 1., 0., 0., 0., 0.]),
 tensor([0., 1., 1., 0., 1., 0., 0.]),
 tensor([0., 1., 1., 1., 0., 0., 0.])]